# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import random
from data import Data, STATS_COLUMNS
import model
import elo

torch.manual_seed(20250222)
random.seed(20250222)
np.random.seed(20250222)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
dataset = Data(batch_size=100)

dataset.games.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,...,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000
mean,2015.575896,70.371624,2101.677721,74.190697,2099.323680,61.205668,0.061738,26.180557,57.072441,6.919323,...,19.266279,11.437051,16.819468,10.801152,21.937435,11.218625,15.191485,6.454031,2.849489,18.836014
std,6.073611,36.089915,986.770174,11.403172,989.989621,11.373177,0.287218,4.809783,7.820486,3.168482,...,6.327385,5.238806,6.985267,4.417465,4.706791,3.764948,5.029754,2.984269,2.036512,4.586616
min,2003.000000,0.000000,1101.000000,30.000000,1101.000000,11.000000,0.000000,9.000000,26.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2011.000000,38.000000,1261.000000,66.000000,1254.000000,54.000000,0.000000,23.000000,52.000000,5.000000,...,15.000000,8.000000,12.000000,8.000000,19.000000,9.000000,12.000000,4.000000,1.000000,16.000000
50%,2016.000000,74.000000,1413.000000,74.000000,1407.000000,61.000000,0.000000,26.000000,57.000000,7.000000,...,19.000000,11.000000,16.000000,10.000000,22.000000,11.000000,15.000000,6.000000,3.000000,19.000000
75%,2021.000000,101.000000,3245.000000,81.000000,3246.000000,69.000000,0.000000,29.000000,62.000000,9.000000,...,23.000000,15.000000,21.000000,14.000000,25.000000,14.000000,18.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,3480.000000,149.000000,3480.000000,144.000000,6.000000,58.000000,113.000000,30.000000,...,80.000000,48.000000,65.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The x's will be the indexes of two team IDs and program IDs, the y's will include and indicator of who won at the game stats

## Predicting Stats
The first model we build will predict game statistics. There is more information encapsulated in the box scores than simply wins and losses so we will use this to build up embeddings that are predictive about how the games will play out

In [3]:
stats_train_loader, stats_validation_loader = dataset.train_test_data(output_stats=True, cache="stats", train_size=0.99)

Generating train dataset
Generating test dataset


In [4]:
stats_model = model.StatsModel(program_embedding=512, team_embedding=512,
                               num_programs=len(dataset.programs), num_teams=len(dataset.teams),
                               model_sizes=(512, 256), dropout=0.5).to(device)

In [5]:
model.train(stats_train_loader, stats_validation_loader, stats_model, name="stats_model", learning_rate=0.001, use_cache=True)

Epoch   0: Train Loss=316.64741412, Test Loss=318.32854130
Epoch   1: Train Loss=281.47825186, Test Loss=282.63592741
Epoch   2: Train Loss=205.28821859, Test Loss=206.62036688
Epoch   3: Train Loss=128.02987381, Test Loss=129.28279478
Epoch   4: Train Loss=44.00895754, Test Loss=45.32972868
Epoch   5: Train Loss=42.40044527, Test Loss=43.81982422
Epoch   6: Train Loss=37.60805004, Test Loss=38.96179271
Epoch   7: Train Loss=35.65429696, Test Loss=36.94423061
Epoch   8: Train Loss=34.56044473, Test Loss=35.87356506
Epoch   9: Train Loss=33.59118474, Test Loss=34.89429369
Epoch  10: Train Loss=34.43326222, Test Loss=35.80883636
Epoch  11: Train Loss=33.64899278, Test Loss=35.03000489
Epoch  12: Train Loss=33.24701610, Test Loss=34.61892038
Epoch  13: Train Loss=32.91768280, Test Loss=34.32396312
Epoch  14: Train Loss=32.15074945, Test Loss=33.59918143
Epoch  15: Train Loss=32.70508950, Test Loss=34.12729226
Epoch  16: Train Loss=33.33539376, Test Loss=34.80816520
Epoch  17: Train Loss=3

## The Model
Now we use the embeddings developed in the first model to build a model to predict which team will win, which is the thing we are, in the end trying to do.

In [6]:
result_train_loader, result_validation_loader = dataset.train_test_data(output_stats=False, cache="result", train_size=0.99, seed=20250320)

Generating train dataset
Generating test dataset


In [7]:
result_model = model.Model(matchup=stats_model.matchup, model_sizes=(384, 32), dropout=0.5).to(device)
result_model.matchup.freeze()

## Training the model

Train the model

In [8]:
model.train(result_train_loader, result_validation_loader, result_model, name="result_model", learning_rate=0.0001, use_cache=True)

Epoch   0: Train Loss=0.24967134, Test Loss=0.24968268
Epoch   1: Train Loss=0.22107351, Test Loss=0.22170225
Epoch   2: Train Loss=0.16327772, Test Loss=0.16652930
Epoch   3: Train Loss=0.16229226, Test Loss=0.16582584
Epoch   4: Train Loss=0.16132405, Test Loss=0.16504818
Epoch   5: Train Loss=0.16305041, Test Loss=0.16668884
Epoch   6: Train Loss=0.16218443, Test Loss=0.16603052
Epoch   7: Train Loss=0.16263446, Test Loss=0.16645418
Epoch   8: Train Loss=0.16183125, Test Loss=0.16583480
Epoch   9: Train Loss=0.16200305, Test Loss=0.16590126
Best Loss: 0.165048


In [9]:
model.print_results(result_validation_loader, result_model, label="Validation Results")

Validation Results: Accuracy=75.52, Loss=0.165048


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [10]:
tourney_dataset = dataset.gen_dataset(dataset.tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=500, shuffle=True)

In [11]:
model.print_results(tourney_loader, result_model, label="Tournament Results")

Tournament Results: Accuracy=73.75, Loss=0.169934


When it comes to tournament results we get a little worse. The lower result is likely due to teams having increased pairity.

In [12]:
stage1_loader = dataset.tourney_data(after=2021)
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=71.94, Loss=0.176540


Train with early torney data

In [13]:
tourney_df = dataset.tourney[dataset.tourney.Season < 2021]

tourney_train_df, tourney_validation_df = train_test_split(tourney_df, train_size=0.8)
tourney_train_data = dataset.gen_dataset(tourney_train_df)
tourney_validation_data = dataset.gen_dataset(tourney_validation_df)

tourney_train_loader = DataLoader(tourney_train_data, batch_size=500)
tourney_validation_loader = DataLoader(tourney_validation_data, batch_size=500)

In [14]:
result_model.matchup.freeze()

In [15]:
model.train(tourney_train_loader, tourney_validation_loader, result_model, name="tuned_model", learning_rate=0.0001, use_cache=True)

Epoch   0: Train Loss=0.17060390, Test Loss=0.18130773
Epoch   1: Train Loss=0.16769923, Test Loss=0.17772005
Epoch   2: Train Loss=0.16586363, Test Loss=0.17577651
Epoch   3: Train Loss=0.16557412, Test Loss=0.17573301
Epoch   4: Train Loss=0.16572047, Test Loss=0.17601828
Epoch   5: Train Loss=0.16557112, Test Loss=0.17604638
Epoch   6: Train Loss=0.16523994, Test Loss=0.17598317
Epoch   7: Train Loss=0.16491247, Test Loss=0.17581289
Epoch   8: Train Loss=0.16468226, Test Loss=0.17560270
Epoch   9: Train Loss=0.16458113, Test Loss=0.17546044
Epoch  10: Train Loss=0.16443325, Test Loss=0.17543228
Epoch  11: Train Loss=0.16428542, Test Loss=0.17547675
Epoch  12: Train Loss=0.16413970, Test Loss=0.17552954
Epoch  13: Train Loss=0.16392659, Test Loss=0.17558169
Epoch  14: Train Loss=0.16372619, Test Loss=0.17559747
Epoch  15: Train Loss=0.16343430, Test Loss=0.17562011
Best Loss: 0.175432


In [16]:
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=71.75, Loss=0.176297


### Performance by year


In [17]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(year=season)
    model.print_results(loader, result_model, label=f"{season} Tournament")

2003 Tournament: Accuracy=78.12, Loss=0.167594
2004 Tournament: Accuracy=70.31, Loss=0.181137
2005 Tournament: Accuracy=75.00, Loss=0.174490
2006 Tournament: Accuracy=71.09, Loss=0.204490
2007 Tournament: Accuracy=77.34, Loss=0.177735
2008 Tournament: Accuracy=78.91, Loss=0.166241
2009 Tournament: Accuracy=78.91, Loss=0.154354
2010 Tournament: Accuracy=70.87, Loss=0.170099
2011 Tournament: Accuracy=75.38, Loss=0.170091
2012 Tournament: Accuracy=76.54, Loss=0.159456
2013 Tournament: Accuracy=71.92, Loss=0.174500
2014 Tournament: Accuracy=74.23, Loss=0.162976
2015 Tournament: Accuracy=81.15, Loss=0.142555
2016 Tournament: Accuracy=76.15, Loss=0.169860
2017 Tournament: Accuracy=79.23, Loss=0.153805
2018 Tournament: Accuracy=68.46, Loss=0.184314
2019 Tournament: Accuracy=76.54, Loss=0.149410
2021 Tournament: Accuracy=72.09, Loss=0.178727
2022 Tournament: Accuracy=73.88, Loss=0.173136
2023 Tournament: Accuracy=69.03, Loss=0.188523
2024 Tournament: Accuracy=72.01, Loss=0.164893


Breaking out by league

In [18]:
for season in dataset.tourney.Season.unique():
    for league in dataset.tourney[dataset.tourney.Season == season].League.unique():
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, result_model, label=f"{season} {league} Tournament")

2003 M Tournament: Accuracy=78.12, Loss=0.167594
2004 M Tournament: Accuracy=70.31, Loss=0.181137
2005 M Tournament: Accuracy=75.00, Loss=0.174490
2006 M Tournament: Accuracy=71.09, Loss=0.204490
2007 M Tournament: Accuracy=77.34, Loss=0.177735
2008 M Tournament: Accuracy=78.91, Loss=0.166241
2009 M Tournament: Accuracy=78.91, Loss=0.154354
2010 M Tournament: Accuracy=67.19, Loss=0.201222
2010 W Tournament: Accuracy=74.60, Loss=0.138482
2011 M Tournament: Accuracy=70.15, Loss=0.216175
2011 W Tournament: Accuracy=80.95, Loss=0.121081
2012 M Tournament: Accuracy=70.15, Loss=0.187989
2012 W Tournament: Accuracy=83.33, Loss=0.129110
2013 M Tournament: Accuracy=66.42, Loss=0.199719
2013 W Tournament: Accuracy=77.78, Loss=0.147679
2014 M Tournament: Accuracy=66.42, Loss=0.199702
2014 W Tournament: Accuracy=82.54, Loss=0.123919
2015 M Tournament: Accuracy=78.36, Loss=0.159907
2015 W Tournament: Accuracy=84.13, Loss=0.124101
2016 M Tournament: Accuracy=76.12, Loss=0.186621
2016 W Tournament: A

## Inspect the model
First what are the sizes of the smallest input and output weights

In [19]:
print(f"Program embedding min: {result_model.matchup.program_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"Team embedding min: {result_model.matchup.team_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {result_model.fc3.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Program embedding min: 2.373395
Team embedding min: 3.467458
FC min: 0.015503


Calculate the average gradient for each input feature

In [20]:
result_model.matchup.unfreeze()

In [21]:
program_weights, team_weights, stats_weights = model.feature_eval(result_model, tourney_loader)

In [22]:
program_weights.abs().sum().item(), team_weights.abs().sum().item()

(0.0003097765729762614, 0.00029842532239854336)

In [23]:
print(f"Year:\t{stats_weights[0]:>4f}")
print(f"Game:\t{stats_weights[1]:>4f}")
print(f"League:\t{stats_weights[2]:>4f}")

Year:	-0.000077
Game:	0.000261
League:	0.004353


## Save the model

In [24]:
torch.save(result_model.state_dict(), 'model.pth')

## Moderated model

Moderate a model by pushing it towards 0.5

In [25]:
moderated = model.ModeratedModel(result_model, 0.75)
model.print_results(stage1_loader, moderated, label="Moderated Model")

Moderated Model: Accuracy=71.75, Loss=0.181928


In [26]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(season)
    model.print_results(loader, moderated, label=f"{season} Tournament")

2003 Tournament: Accuracy=78.12, Loss=0.176851
2004 Tournament: Accuracy=70.31, Loss=0.187284
2005 Tournament: Accuracy=75.00, Loss=0.180904
2006 Tournament: Accuracy=71.09, Loss=0.206225
2007 Tournament: Accuracy=77.34, Loss=0.185758
2008 Tournament: Accuracy=78.91, Loss=0.176762
2009 Tournament: Accuracy=78.91, Loss=0.167343
2010 Tournament: Accuracy=70.87, Loss=0.177011
2011 Tournament: Accuracy=75.38, Loss=0.176038
2012 Tournament: Accuracy=76.54, Loss=0.169539
2013 Tournament: Accuracy=71.92, Loss=0.180712
2014 Tournament: Accuracy=74.23, Loss=0.171342
2015 Tournament: Accuracy=81.15, Loss=0.155751
2016 Tournament: Accuracy=76.15, Loss=0.176265
2017 Tournament: Accuracy=79.23, Loss=0.163937
2018 Tournament: Accuracy=68.46, Loss=0.186157
2019 Tournament: Accuracy=76.54, Loss=0.160908
2021 Tournament: Accuracy=72.09, Loss=0.182840
2022 Tournament: Accuracy=73.88, Loss=0.180490
2023 Tournament: Accuracy=69.03, Loss=0.191795
2024 Tournament: Accuracy=72.01, Loss=0.172620


It underperforms even in years where there were upsets.

## Dig into 2023 results

In [27]:
loader = dataset.tourney_data(2023)

x, y = loader.dataset.tensors

preds = result_model(x.to(device))

In [28]:
t_2023 = pd.DataFrame({'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                       'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                       'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                       'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                       'actual': y.reshape(-1),
                       'predicted': preds.cpu().detach().numpy().reshape(-1)}).iloc[:67]

In [29]:
t_2023[t_2023.predicted < 0.5].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted
23,F Dickinson,Purdue,1192,1345,1.0,0.021932
15,Princeton,Arizona,1343,1112,1.0,0.131119
8,Furman,Virginia,1202,1438,1.0,0.200075
53,FL Atlantic,Tennessee,1194,1397,1.0,0.278538
39,Princeton,Missouri,1343,1281,1.0,0.301827
57,Miami FL,Houston,1274,1222,1.0,0.335080
58,San Diego St,Alabama,1361,1104,1.0,0.341491
50,Michigan St,Marquette,1277,1266,1.0,0.388995
14,Penn St,Texas A&M,1336,1401,1.0,0.394073
37,Arkansas,Kansas,1116,1242,1.0,0.415158


The biggest thing in this season were the huge upsets in the first round. Purdue was a number one seed and lost which I only gave a 1% chance to happen. Arizona and Virginia were number 2 seeds and lost which I gave 13% and 17% chances of happening respectively.

In [30]:
t_2023['Upset'] = [dataset.upset(2023, winner, loser) for (winner, loser) in zip(t_2023['winner'], t_2023['loser'])]

In [31]:
t_2023[t_2023.Upset].predicted.mean()

np.float64(0.37878499330884907)

On average the upsets had a 37% chance of happening

In [32]:
t_2023[t_2023.Upset & (t_2023.predicted >= 0.5)].sort_values('predicted', ascending=False)

,winner_name,loser_name,winner,loser,actual,predicted,Upset
60,Connecticut,Gonzaga,1163,1211,1.0,0.568763,True
45,Creighton,Baylor,1166,1124,1.0,0.554117,True
49,Miami FL,Indiana,1274,1231,1.0,0.505944,True


I correctly predicted 4 upsets, though all were closely ranked

In [33]:
t_2023[~t_2023.Upset & (t_2023.predicted < 0.5)].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted,Upset
12,Missouri,Utah St,1281,1429,1.0,0.424303,False
63,San Diego St,Creighton,1361,1166,1.0,0.425957,False
5,Arkansas,Illinois,1116,1228,1.0,0.461555,False
0,Pittsburgh,Mississippi St,1338,1280,1.0,0.472791,False
55,Kansas St,Michigan St,1243,1277,1.0,0.481324,False
59,Texas,Xavier,1400,1462,1.0,0.483533,False
48,Kansas St,Kentucky,1243,1246,1.0,0.487210,False


I also incorrectly predicted 5 upsets

Looking at all the tourneys

In [34]:
x, y = tourney_loader.dataset.tensors
preds = result_model(x.to(device))
tourney_df = pd.DataFrame({'season': x[:,4].tolist(),
                           'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                           'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                           'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                           'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                           'actual': y.reshape([-1]),
                           'predicted': np.array(preds.tolist()).reshape([-1])})
tourney_df = tourney_df[tourney_df.actual == 1.0]
tourney_df['Upset'] = [dataset.upset(season, winner, loser) for (winner, loser, season)
                       in zip(tourney_df['winner'], tourney_df['loser'], tourney_df['season'])]

In [35]:
len(tourney_df[tourney_df.Upset & (tourney_df.predicted >= 0.5) & (tourney_df.season > 2020)].sort_values('predicted', ascending=False))

29

In [36]:
len(tourney_df[~tourney_df.Upset & (tourney_df.predicted < 0.5) & (tourney_df.season > 2020)].sort_values('predicted'))

39

Overall I predicted 25 upsets correctly, and 36 incorrectly

## Predicting by seeds
What if I predict just using the seeds?

In [37]:
odds = dataset.odds_by_seed_diff(before=2021)

In [38]:
dataset.tourney_df(after=2021).SeedDiff.map(lambda x: odds[x]**2).mean()

np.float64(0.18527460145235355)

This results in a test Brier score of about 0.185.

## Using Elo
I can also compute Elo rankings

In [39]:
elo_model = elo.EloModel(dataset, k=50)
model.print_results(stage1_loader, elo_model, label="Elo Model")

Elo Model: Accuracy=73.45, Loss=0.178886


It is similar to our model

## Hybrid Model
Building a model using the neural net and seeds

In [40]:
seed_model = model.SeedModel(dataset)
model.print_results(stage1_loader, seed_model, label="Seed Model")
model.print_results(stage1_loader, elo_model, label="Elo Model")
model.print_results(stage1_loader, result_model, label="Base NN")

Seed Model: Accuracy=72.03, Loss=0.184022
Elo Model: Accuracy=73.45, Loss=0.178886
Base NN: Accuracy=71.75, Loss=0.176297


In [41]:
hybrid = model.HybridModel([result_model, seed_model, elo_model], [0.7, 0.2, 0.1])

model.print_results(stage1_loader, hybrid, label="Hybrid Model")

Hybrid Model: Accuracy=73.63, Loss=0.175054


They hybrid model outperforms all individual models

In [42]:
for season in range(2021, 2025):
    for league in ['M', 'W']:
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, hybrid, label=f"{season} {league} Tournament")

2021 M Tournament: Accuracy=67.42, Loss=0.210846
2021 W Tournament: Accuracy=77.78, Loss=0.147458
2022 M Tournament: Accuracy=72.39, Loss=0.202837
2022 W Tournament: Accuracy=78.36, Loss=0.144246
2023 M Tournament: Accuracy=67.91, Loss=0.204828
2023 W Tournament: Accuracy=76.87, Loss=0.166638
2024 M Tournament: Accuracy=64.93, Loss=0.191515
2024 W Tournament: Accuracy=83.58, Loss=0.130950


## Generating the submission file
### Phase 2

Write the results

In [43]:
model.gen_submission(hybrid, dataset, fname="hybrid.csv")

In [44]:
model.gen_submission(result_model, dataset, fname="base.csv")

## Generate a bracket

With the hybrid model

In [45]:
season, league = 2025, 'M'

In [46]:
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.986675
R1W2    1104         Alabama  0.904333
R1W3    1458       Wisconsin  0.896732
R1W4    1112         Arizona  0.838286
R1W5    1332          Oregon  0.755051
R1W6    1140             BYU  0.551006
R1W7    1388    St Mary's CA  0.629147
R1W8    1124          Baylor  0.502033
R1X1    1222         Houston  0.974564
R1X2    1397       Tennessee  0.945926
R1X3    1246        Kentucky  0.879722
R1X4    1345          Purdue  0.830151
R1X5    1155         Clemson  0.812503
R1X6    1228        Illinois  0.622264
R1X7    1417            UCLA  0.569426
R1X8    1211         Gonzaga  0.611822
R1Y1    1120          Auburn  0.988184
R1Y2    1277     Michigan St  0.936307
R1Y3    1235         Iowa St  0.856116
R1Y4    1401       Texas A&M  0.772015
R1Y5    1276        Michigan  0.632767
R1Y6    1314  North Carolina  0.548449
R1Y7    1266       Marquette  0.592510
R1Y8    1257      Louisvi

In [47]:
model.tiebreaker(bracket, stats_model, dataset, season, league)

140.15078814263353

And with the base model

In [48]:
base_bracket = model.gen_bracket(dataset, 2025, 'M', result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(base_bracket)

      Winner        TeamName         P
Slot                                  
R1W1    1181            Duke  0.987066
R1W2    1104         Alabama  0.885273
R1W3    1458       Wisconsin  0.883602
R1W4    1112         Arizona  0.833511
R1W5    1332          Oregon  0.749130
R1W6    1140             BYU  0.506145
R1W7    1388    St Mary's CA  0.608083
R1W8    1124          Baylor  0.508905
R1X1    1222         Houston  0.968624
R1X2    1397       Tennessee  0.941186
R1X3    1246        Kentucky  0.861590
R1X4    1345          Purdue  0.817398
R1X5    1155         Clemson  0.820830
R1X6    1228        Illinois  0.621261
R1X7    1417            UCLA  0.548249
R1X8    1211         Gonzaga  0.619506
R1Y1    1120          Auburn  0.988145
R1Y2    1277     Michigan St  0.928447
R1Y3    1235         Iowa St  0.823789
R1Y4    1401       Texas A&M  0.745669
R1Y5    1276        Michigan  0.578778
R1Y6    1314  North Carolina  0.607493
R1Y7    1266       Marquette  0.576482
R1Y8    1257      Louisvi

In [49]:
model.tiebreaker(base_bracket, stats_model, dataset, season, league)

140.15078814263353

Women's bracket

In [50]:
season, league = 2025, 'W'
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    3376  South Carolina  0.987654
R1W2    3181            Duke  0.961906
R1W3    3314  North Carolina  0.891423
R1W4    3268        Maryland  0.853042
R1W5    3104         Alabama  0.737904
R1W6    3452   West Virginia  0.756078
R1W7    3435      Vanderbilt  0.674209
R1W8    3428            Utah  0.503113
R1X1    3400           Texas  0.987813
R1X2    3395             TCU  0.973414
R1X3    3323      Notre Dame  0.965992
R1X4    3326         Ohio St  0.848856
R1X5    3397       Tennessee  0.825428
R1X6    3276        Michigan  0.601366
R1X7    3257      Louisville  0.540839
R1X8    3166       Creighton  0.548263
R1Y1    3417            UCLA  0.988275
R1Y2    3301        NC State  0.944486
R1Y3    3261             LSU  0.936295
R1Y4    3124          Baylor  0.868438
R1Y5    3279     Mississippi  0.717460
R1Y6    3199      Florida St  0.748671
R1Y7    3277     Michigan St  0.692537
R1Y8    3210    Georgia T

In [51]:
bracket = model.gen_bracket(dataset, season, league, result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName', 'P']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName         P
Slot                                  
R1W1    3376  South Carolina  0.987141
R1W2    3181            Duke  0.962766
R1W3    3314  North Carolina  0.883093
R1W4    3268        Maryland  0.846213
R1W5    3104         Alabama  0.740186
R1W6    3452   West Virginia  0.805024
R1W7    3435      Vanderbilt  0.689356
R1W8    3231         Indiana  0.500175
R1X1    3400           Texas  0.987096
R1X2    3395             TCU  0.978804
R1X3    3323      Notre Dame  0.974371
R1X4    3326         Ohio St  0.836229
R1X5    3397       Tennessee  0.849102
R1X6    3276        Michigan  0.591942
R1X7    3304        Nebraska  0.506392
R1X8    3166       Creighton  0.549659
R1Y1    3417            UCLA  0.988329
R1Y2    3301        NC State  0.937532
R1Y3    3261             LSU  0.935682
R1Y4    3124          Baylor  0.872859
R1Y5    3279     Mississippi  0.705458
R1Y6    3199      Florida St  0.774010
R1Y7    3277     Michigan St  0.707213
R1Y8    3210    Georgia T

In [73]:
model.tiebreaker(bracket, stats_model, dataset, season, league)

135.42561193166972

## Elo Rankings
### Mens

In [52]:
elo_rankings = [(elo_model.ratings[dataset.teamMapping[(team, 2025)]],team) 
                for team in dataset.seeds.loc[2025].loc['M'].index]
for ranking, team in sorted(elo_rankings, reverse=True):
    print(f"{dataset.all_teams.loc[team].TeamName} ({team}): {ranking:.2f}")

Houston (1222): 1770.81
Florida (1196): 1699.45
Duke (1181): 1670.73
St John's (1385): 1663.18
Auburn (1120): 1659.61
Tennessee (1397): 1622.20
Michigan St (1277): 1620.11
Alabama (1104): 1612.31
BYU (1140): 1565.44
Connecticut (1163): 1557.59
Texas Tech (1403): 1545.10
Michigan (1276): 1538.41
Wisconsin (1458): 1536.35
Creighton (1166): 1534.52
Iowa St (1235): 1534.19
Arizona (1112): 1525.12
Maryland (1268): 1521.87
St Mary's CA (1388): 1518.80
Gonzaga (1211): 1517.88
Clemson (1155): 1515.53
Oregon (1332): 1507.60
Kentucky (1246): 1503.35
Illinois (1228): 1501.04
Purdue (1345): 1497.22
Colorado St (1161): 1486.80
Texas A&M (1401): 1484.07
Marquette (1266): 1483.97
Louisville (1257): 1481.30
Memphis (1272): 1480.30
Kansas (1242): 1453.77
North Carolina (1314): 1449.45
Drake (1179): 1447.69
UCLA (1417): 1446.64
Mississippi (1279): 1445.07
New Mexico (1307): 1432.34
Xavier (1462): 1430.93
VCU (1433): 1424.17
Baylor (1124): 1422.12
Utah St (1429): 1418.39
Arkansas (1116): 1405.07
Oklahoma

### Women's

In [53]:
elo_rankings = [(elo_model.ratings[dataset.teamMapping[(team, 2025)]],team) 
                for team in dataset.seeds.loc[2025].loc['W'].index]
for ranking, team in sorted(elo_rankings, reverse=True):
    print(f"{dataset.all_teams.loc[team].TeamName} ({team}): {ranking:.2f}")

South Carolina (3376): 2051.64
Texas (3400): 1908.29
UCLA (3417): 1895.33
Connecticut (3163): 1881.40
USC (3425): 1874.32
Notre Dame (3323): 1788.93
Duke (3181): 1774.16
NC State (3301): 1763.91
LSU (3261): 1748.59
Ohio St (3326): 1712.88
TCU (3395): 1687.75
Oklahoma (3328): 1680.38
Creighton (3166): 1666.87
Iowa (3234): 1666.54
North Carolina (3314): 1655.81
Baylor (3124): 1650.63
Maryland (3268): 1644.01
Louisville (3257): 1613.82
Michigan (3276): 1607.29
Mississippi (3279): 1604.18
Florida St (3199): 1593.95
West Virginia (3452): 1579.69
Tennessee (3397): 1574.51
Indiana (3231): 1568.17
California (3143): 1558.56
S Dakota St (3355): 1553.67
Iowa St (3235): 1539.89
Michigan St (3277): 1538.06
Kansas St (3243): 1537.08
Alabama (3104): 1536.62
Kentucky (3246): 1532.42
Oklahoma St (3329): 1532.40
Utah (3428): 1521.25
Nebraska (3304): 1515.99
Vanderbilt (3435): 1505.10
Illinois (3228): 1490.10
Columbia (3162): 1471.12
FGCU (3195): 1463.76
Princeton (3343): 1457.45
Richmond (3350): 1448.9

## Predictions


In [54]:
models = {'hybrid': hybrid, 'base': result_model, 'moderated': moderated, 'elo': elo_model, 'seed': seed_model}

## Mens
### Round 1

In [55]:
mens_round_1 = [[1106,1120,'M'],[1166,1257,'M'],[1276,1471,'M'],[1401,1463,'M'],[1279,1314,'M'],[1235,1252,'M'],[1266,1307,'M'],[1136,1280,'M'],#South
                [1181,1291,'M'],[1140,1433,'M'],[1251,1332,'M'],[1103,1112,'M'],[1285,1458,'M'],[1124,1277,'M'],[1388,1435,'M'],[1104,1352,'M'],#East
                [1196,1313,'M'],[1163,1328,'M'],[1161,1272,'M'],[1213,1268,'M'],[1179,1281,'M'],[1403,1423,'M'],[1116,1242,'M'],[1303,1385,'M'],#West
                [1188,1222,'M'],[1208,1211,'M'],[1155,1270,'M'],[1219,1345,'M'],[1228,1462,'M'],[1246,1407,'M'],[1417,1429,'M'],[1397,1459,'M']]#Midwest


In [56]:
model.gen_round_df(dataset, mens_round_1, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,Alabama St,Auburn,1106,1120,0.011816,0.011855,0.133891,0.006599,0.014286
1,Creighton,Louisville,1166,1257,0.439627,0.417106,0.437829,0.576003,0.450262
2,Michigan,UC San Diego,1276,1471,0.632767,0.578778,0.559083,0.785001,0.745614
3,Texas A&M,Yale,1401,1463,0.772015,0.745669,0.684252,0.791378,0.854545
4,Mississippi,North Carolina,1279,1314,0.451551,0.392507,0.419380,0.493702,0.637131
5,Iowa St,Lipscomb,1235,1252,0.856116,0.823789,0.742842,0.926216,0.934211
6,Marquette,New Mexico,1266,1307,0.592510,0.576482,0.557361,0.573761,0.657980
7,Bryant,Mississippi St,1136,1280,0.163941,0.143176,0.232382,0.128402,0.254386
8,Duke,Mt St Mary's,1181,1291,0.986902,0.988085,0.866064,0.980994,0.985714
9,BYU,VCU,1140,1433,0.551006,0.506145,0.504609,0.692790,0.637131


### Round 2

In [57]:
mens_round_2 = [[1120,1166,'M'],[1276,1401,'M'],[1235,1279,'M'],[1277,1307,'M'],#South
                [1124,1181,'M'],[1112,1332,'M'],[1140,1458,'M'],[1104,1388,'M'],#East
                [1163,1196,'M'],[1161,1268,'M'],[1179,1403,'M'],[1116,1385,'M'],#West
                [1211,1222,'M'],[1270,1345,'M'],[1228,1246,'M'],[1397,1417,'M']]#MidWest
model.gen_round_df(dataset, mens_round_2, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,Auburn,Creighton,1120,1166,0.698428,0.678334,0.633751,0.672624,0.781659
1,Michigan,Texas A&M,1276,1401,0.506521,0.512445,0.509334,0.577568,0.450262
2,Iowa St,Mississippi,1235,1279,0.636385,0.631767,0.598826,0.625520,0.657980
3,Michigan St,New Mexico,1277,1307,0.704115,0.675881,0.631910,0.746665,0.781659
4,Baylor,Duke,1124,1181,0.173624,0.158090,0.243568,0.192924,0.218341
5,Arizona,Oregon,1112,1332,0.538209,0.536773,0.527579,0.525203,0.549738
6,BYU,Wisconsin,1140,1458,0.463046,0.486380,0.489785,0.541762,0.342020
7,Alabama,St Mary's CA,1104,1388,0.577339,0.552532,0.539399,0.631405,0.637131
8,Connecticut,Florida,1163,1196,0.281031,0.285008,0.338756,0.306479,0.254386
9,Colorado St,Maryland,1161,1268,0.325260,0.338030,0.378523,0.449708,0.218341


### Round 3

In [71]:
mens_round_3 = [[1120,1276,'M'],[1279,1277,'M'],
                [1181,1112,'M'],[1140,1104,'M'],
                [1196,1268,'M'],[1403,1116,'M'],
                [1222,1345,'M'],[1246,1397,'M']]
model.gen_round_df(dataset, mens_round_3, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,Auburn,Michigan,1120,1276,0.685225,0.667971,0.625978,0.667682,0.754386
1,Mississippi,Michigan St,1279,1277,0.304395,0.326469,0.369851,0.267441,0.245614
2,Duke,Arizona,1181,1112,0.740351,0.769924,0.702443,0.698083,0.657980
3,BYU,Alabama,1140,1104,0.344385,0.359952,0.394964,0.432960,0.245614
4,Florida,Maryland,1196,1268,0.624384,0.598924,0.574193,0.735415,0.657980
5,Texas Tech,Arkansas,1403,1116,0.665198,0.638499,0.603874,0.691262,0.745614
6,Houston,Purdue,1222,1345,0.622226,0.582546,0.561909,0.828476,0.657980
7,Kentucky,Tennessee,1246,1397,0.434452,0.444095,0.458072,0.335326,0.450262


## Womens
### Round 1

In [58]:
womens_round_1 = [[3380,3417,'W'],[3210,3350,'W'],[3123,3279,'W'],[3124,3213,'W'],[3199,3206,'W'],[3261,3361,'W'],[3217,3277,'W'],[3301,3436,'W'],#Reg1
                  [3376,3399,'W'],[3231,3428,'W'],[3104,3453,'W'],[3268,3313,'W'],[3162,3452,'W'],[3314,3333,'W'],[3332,3435,'W'],[3181,3250,'W'],#Reg2
                  [3400,3456,'W'],[3166,3228,'W'],[3378,3397,'W'],[3286,3326,'W'],[3235,3276,'W'],[3323,3372,'W'],[3257,3304,'W'],[3192,3395,'W'],#Reg3
                  [3422,3425,'W'],[3143,3280,'W'],[3193,3243,'W'],[3246,3251,'W'],[3234,3293,'W'],[3195,3328,'W'],[3329,3355,'W'],[3117,3163,'W']]#Reg4
model.gen_round_df(dataset, womens_round_1, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,Southern Univ,UCLA,3380,3417,0.005575,0.003411,0.127558,0.003304,0.014286
1,Georgia Tech,Richmond,3210,3350,0.554674,0.594257,0.570693,0.486416,0.450262
2,Ball St,Mississippi,3123,3279,0.282540,0.294542,0.345906,0.254837,0.254386
3,Baylor,Grand Canyon,3124,3213,0.868438,0.872859,0.779644,0.865274,0.854545
4,Florida St,George Mason,3199,3206,0.748671,0.774010,0.705507,0.794384,0.637131
5,LSU,San Diego St,3261,3361,0.936295,0.935682,0.826762,0.944748,0.934211
6,Harvard,Michigan St,3217,3277,0.307463,0.292787,0.344590,0.341080,0.342020
7,NC State,Vermont,3301,3436,0.944486,0.937532,0.828149,0.982140,0.950000
8,South Carolina,Tennessee Tech,3376,3399,0.987654,0.987141,0.865356,0.995125,0.985714
9,Indiana,Utah,3231,3428,0.496887,0.500175,0.500131,0.567121,0.450262


### Round 2

In [68]:
womens_round_2 = [[3417,3350,'W'],[3279,3124,'W'],[3199,3261,'W'],[3277,3301,'W'],#Regional 1
                  [3376,3231,'W'],[3104,3268,'W'],[3452,3314,'W'],[3332,3181,'W'],#Regional 2
                  [3400,3228,'W'],[3397,3326,'W'],[3276,3323,'W'],[3257,3395,'W'],#Regional 3
                  [3425,3280,'W'],[3243,3246,'W'],[3234,3328,'W'],[3355,3163,'W']]#Regional 4
model.gen_round_df(dataset, womens_round_2, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,UCLA,Richmond,3417,3350,0.863367,0.887654,0.790740,0.928863,0.745614
1,Mississippi,Baylor,3279,3124,0.393672,0.371809,0.403856,0.433540,0.450262
2,Florida St,LSU,3199,3261,0.372434,0.392746,0.419559,0.291077,0.342020
3,Michigan St,NC State,3277,3301,0.442899,0.498444,0.498833,0.214143,0.362869
4,South Carolina,Indiana,3376,3231,0.884868,0.906229,0.804672,0.941755,0.781659
5,Alabama,Maryland,3104,3268,0.436702,0.445188,0.458891,0.350185,0.450262
6,West Virginia,North Carolina,3452,3314,0.539966,0.617636,0.588227,0.392174,0.342020
7,Oregon,Duke,3332,3181,0.172101,0.165790,0.249343,0.123801,0.218341
8,Texas,Illinois,3400,3228,0.839811,0.855644,0.766733,0.917378,0.745614
9,Tennessee,Ohio St,3397,3326,0.456116,0.478554,0.483915,0.310763,0.450262


### Round 3

In [72]:
womens_round_3 = [[3417,3279,'W'],[3261,3301,'W'],
                  [3376,3268,'W'],[3314,3181,'W'],
                  [3400,3397,'W'],[3323,3395,'W'],
                  [3425,3243,'W'],[3328,3163,'W']]
model.gen_round_df(dataset, womens_round_3, models)

,team1Name,team2Name,team1ID,team2ID,hybrid,base,moderated,elo,seed
0,UCLA,Mississippi,3417,3279,0.798853,0.805340,0.729005,0.842375,0.754386
1,LSU,NC State,3261,3301,0.529717,0.559811,0.544858,0.477964,0.450262
2,South Carolina,Maryland,3376,3268,0.843976,0.887306,0.790480,0.912656,0.657980
3,North Carolina,Duke,3314,3181,0.332650,0.298571,0.348928,0.335977,0.450262
4,Texas,Tennessee,3400,3397,0.730541,0.703478,0.652608,0.872293,0.754386
5,Notre Dame,TCU,3323,3395,0.648063,0.705498,0.654124,0.641622,0.450262
6,USC,Kansas St,3425,3243,0.598794,0.514954,0.511215,0.874494,0.754386
7,Oklahoma,Connecticut,3328,3163,0.271492,0.225032,0.293774,0.239175,0.450262


## Actual Results

In [69]:
actual_games = np.array(
    [dataset.matchup(winner, loser, 2025, league) for winner, loser, league in
     [# Mens
         # Round 1
         [1120,1106,'M'],[1166,1257,'M'],[1276,1471,'M'],[1401,1463,'M'],[1279,1314,'M'],[1235,1252,'M'],[1307,1266,'M'],[1280,1136,'M'],#South
         [1181,1291,'M'],[1140,1433,'M'],[1332,1251,'M'],[1112,1103,'M'],[1458,1285,'M'],[1124,1277,'M'],[1388,1435,'M'],[1104,1352,'M'],#East
         [1196,1313,'M'],[1163,1328,'M'],[1161,1272,'M'],[1268,1213,'M'],[1179,1281,'M'],[1403,1423,'M'],[1116,1242,'M'],[1385,1303,'M'],#West
         [1222,1188,'M'],[1211,1208,'M'],[1270,1155,'M'],[1345,1219,'M'],[1228,1462,'M'],[1246,1407,'M'],[1417,1429,'M'],[1397,1459,'M'],#Midwest
         # Round 2
         [1120,1166,'M'],[1276,1401,'M'],[1279,1235,'M'],[1277,1307,'M'],
         [1181,1124,'M'],[1112,1332,'M'],[1140,1458,'M'],[1104,1388,'M'],
         [1196,1163,'M'],[1268,1161,'M'],[1403,1179,'M'],[1116,1385,'M'],
         [1222,1211,'M'],[1345,1270,'M'],[1246,1228,'M'],[1397,1417,'M'],
      # Womens
         # Round 1
         [3417,3380,'W'],[3350,3210,'W'],[3279,3123,'W'],[3124,3213,'W'],[3199,3206,'W'],[3261,3361,'W'],[3277,3217,'W'],[3301,3436,'W'],#Regional 1
         [3376,3399,'W'],[3231,3428,'W'],[3104,3453,'W'],[3268,3313,'W'],[3452,3162,'W'],[3314,3333,'W'],[3332,3435,'W'],[3181,3250,'W'],#Regional 2
         [3400,3456,'W'],[3228,3166,'W'],[3397,3378,'W'],[3326,3286,'W'],[3276,3235,'W'],[3323,3372,'W'],[3257,3304,'W'],[3395,3192,'W'],#Regional 3
         [3425,3422,'W'],[3280,3143,'W'],[3243,3193,'W'],[3246,3251,'W'],[3234,3293,'W'],[3328,3195,'W'],[3355,3329,'W'],[3163,3117,'W'],#Regional 4
         # Round 2
         [3417,3350,'W'],[3279,3124,'W'],[3261,3199,'W'],[3301,3277,'W'],
         [3376,3231,'W'],[3268,3104,'W'],[3314,3452,'W'],[3181,3332,'W'],
         [3400,3228,'W'],[3397,3326,'W'],[3323,3276,'W'],[3395,3257,'W'],
         [3425,3280,'W'],[3243,3243,'W'],[3328,3234,'W'],[3163,3355,'W'],
     ]])
actual_results = np.ones((len(actual_games),1))
actual_ds = TensorDataset(torch.from_numpy(actual_games), torch.from_numpy(actual_results))
actual_loader = DataLoader(actual_ds)
model.print_results(actual_loader, hybrid, label="Actual Hybrid Model")
model.print_results(actual_loader, result_model, label="Actual Base Model")
model.print_results(actual_loader, moderated, label="Actual Moderated Model")
model.print_results(actual_loader, elo_model, label="Actual Elo Model")
model.print_results(actual_loader, seed_model, label="Actual Seeds Model")


Actual Hybrid Model: Accuracy=79.17, Loss=0.132635
Actual Base Model: Accuracy=79.17, Loss=0.138128
Actual Moderated Model: Accuracy=79.17, Loss=0.150548
Actual Elo Model: Accuracy=87.50, Loss=0.120797
Actual Seeds Model: Accuracy=82.29, Loss=0.129946
